## The FineTune/Warm Up option

Let's place ourselves in two possible scenarios. 

1. Let's assume we have run a model and we want to just transfer the learnings (you know...transfer-learning) to another dataset, or simply we have received new data and we do not want to start the training of each component from scratch. Simply, we want to load the pre-trained weights and fine-tune.

2. We just want to "warm up" individual model components individually before the joined training begins.  

This can be done with the `finetune` set of parameters (aliased all as `warmup` parameters). There are 3 fine-tuning routines:

1. Fine-tune all trainable layers at once with a triangular one-cycle learning rate (referred as slanted triangular learning rates in Howard & Ruder 2018)
2. Gradual fine-tuning inspired by the work of [Felbo et al., 2017](https://arxiv.org/abs/1708.00524)
3. Gradual fine-tuning based on the work of [Howard & Ruder 2018](https://arxiv.org/abs/1801.06146)

Currently fine-tunning is only supported without a fully connected head, i.e. if `deephead=None`. In addition, `Felbo` and `Howard` routines only applied, of course, to the `deeptabular`, `deeptext` and `deepimage` models. The `wide` component can also be fine-tuned, but only in an "all at once" mode.

### Fine-tune or warm-up all at once

Here, the model components will be trained for `finetune_epochs` using a triangular one-cycle learning rate (slanted triangular learning rate) ranging from `finetune_max_lr/10` to `finetune_max_lr` (default is 0.01). 10% of the training steps are used to increase the learning rate which then decreases for the remaining 90%. 

Here all trainable layers are fine-tuned.

Let's have a look to one example. 

In [1]:
import numpy as np
import pandas as pd
import torch

from pytorch_widedeep import Trainer
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, TabMlp, TabResnet, WideDeep
from pytorch_widedeep.metrics import Accuracy

In [2]:
df = pd.read_csv('data/adult/adult.csv.zip')
# For convenience, we'll replace '-' with '_'
df.columns = [c.replace("-", "_") for c in df.columns]
# binary target
df['income_label'] = (df["income"].apply(lambda x: ">50K" in x)).astype(int)
df.drop('income', axis=1, inplace=True)
df.head()

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_label
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,0


In [3]:
wide_cols = ['education', 'relationship','workclass','occupation','native_country','gender']
crossed_cols = [('education', 'occupation'), ('native_country', 'occupation')]
cat_embed_cols = [('education',16), ('relationship',8), ('workclass',16), ('occupation',16),('native_country',16)]
continuous_cols = ["age","hours_per_week"]
target_col = 'income_label'

In [4]:
# TARGET
target = df[target_col].values

# WIDE
wide_preprocessor = WidePreprocessor(wide_cols=wide_cols, crossed_cols=crossed_cols)
X_wide = wide_preprocessor.fit_transform(df)

# DEEP
tab_preprocessor = TabPreprocessor(embed_cols=cat_embed_cols, continuous_cols=continuous_cols)
X_tab = tab_preprocessor.fit_transform(df)

In [5]:
wide = Wide(wide_dim=np.unique(X_wide).shape[0], pred_dim=1)
deeptabular = TabMlp(mlp_hidden_dims=[64,32], 
                   column_idx=tab_preprocessor.column_idx,
                   embed_input=tab_preprocessor.embeddings_input,
                   continuous_cols=continuous_cols)
model = WideDeep(wide=wide, deeptabular=deeptabular)

In [6]:
trainer = Trainer(model, objective="binary", metrics=[Accuracy])

Up until here is identical to the code in notebook `03_Binary_Classification_with_Defaults`. Now you can warm up via the warm up parameters

In [7]:
trainer.fit(X_wide=X_wide, X_tab=X_tab, target=target, n_epochs=2, val_split=0.2)

valid: 100%|██████████| 306/306 [00:01<00:00, 174.61it/s, loss=0.365, metrics={'acc': 0.837}] 


In [8]:
trainer.save_model("models_dir/model1.t")

Now time goes by...and we want to fine-tune the model to another, new dataset (for example, a dataset that is identical to the one you used to train the previous model but for another country). 

Here I will use the same dataset just for illustration purposes, but the flow would be identical to that new dataset

In [9]:
trainer2 = Trainer(model_path="models_dir/model1.t", objective="binary", metrics=[Accuracy])

In [10]:
trainer2.fit(X_wide=X_wide, X_tab=X_tab, target=target, finetune=True, finetune_epochs=2, n_epochs=2)

  0%|          | 0/1527 [00:00<?, ?it/s]

Training wide for 2 epochs


  0%|          | 0/1527 [00:00<?, ?it/s]

Training deeptabular for 2 epochs


  0%|          | 0/1527 [00:00<?, ?it/s]

Fine-tuning of individual components completed. Training the whole model for 2 epochs


epoch 2: 100%|██████████| 1527/1527 [00:13<00:00, 113.07it/s, loss=0.333, metrics={'acc': 0.8454}]


Note that, as I describe above, in scenario 2, we can just use this to warm up models before they joined training begins:

In [11]:
wide = Wide(wide_dim=np.unique(X_wide).shape[0], pred_dim=1)
deeptabular = TabMlp(mlp_hidden_dims=[128, 32], 
                   column_idx=tab_preprocessor.column_idx,
                   embed_input=tab_preprocessor.embeddings_input,
                   continuous_cols=continuous_cols)
model = WideDeep(wide=wide, deeptabular=deeptabular)

In [12]:
trainer3 = Trainer(model, objective="binary", metrics=[Accuracy])

In [13]:
trainer3.fit(X_wide=X_wide, X_tab=X_tab, target=target, val_split=0.1, warmup=True, warmup_epochs=2, n_epochs=2)

  0%|          | 0/1374 [00:00<?, ?it/s]

Training wide for 2 epochs


  0%|          | 0/1374 [00:00<?, ?it/s]

Training deeptabular for 2 epochs


  0%|          | 0/1374 [00:00<?, ?it/s]

Fine-tuning of individual components completed. Training the whole model


valid: 100%|██████████| 153/153 [00:00<00:00, 160.45it/s, loss=0.339, metrics={'acc': 0.8451}]


### Fine-tune Gradually: The "felbo"  and the "howard" routines

The Felbo routine can be illustrated as follows:

<p align="center">
  <img width="600" src="../docs/figures/felbo_routine.png">
</p>

**Figure 1.** The figure can be described as follows: fine-tune (or train) the last layer for one epoch using a one cycle triangular learning rate. Then fine-tune the next deeper layer for one epoch, with a learning rate that is a factor of 2.5 lower than the previous learning rate (the 2.5 factor is fixed) while freezing the already warmed up layer(s). Repeat untill all individual layers are warmed. Then warm one last epoch with all warmed layers trainable. The vanishing color gradient in the figure attempts to illustrate the decreasing learning rate. 

Note that this is not identical to the Fine-Tunning routine described in Felbo et al, 2017, this is why I used the word 'inspired'.

The Howard routine can be illustrated as follows:

<p align="center">
  <img width="600" src="../docs/figures/howard_routine.png">
</p>

**Figure 2.** The figure can be described as follows: fine-tune (or train) the last layer for one epoch using a one cycle triangular learning rate. Then fine-tune the next deeper layer for one epoch, with a learning rate that is a factor of 2.5 lower than the previous learning rate (the 2.5 factor is fixed) while keeping the already warmed up layer(s) trainable. Repeat. The vanishing color gradient in the figure attempts to illustrate the decreasing learning rate. 

Note that I write "*fine-tune (or train) the last layer for one epoch [...]*". However, in practice the user will have to specify the order of the layers to be fine-tuned. This is another reason why I wrote that the fine-tune routines I have implemented are **inspired** by the work of Felbo and Howard and not identical to their implemenations.

The `felbo` and `howard` routines can be accessed with via the `fine-tune` parameters.

We need to explicitly indicate 

1. That we want fine-tune

2. The components that we want to individually fine-tune 

3. In case of gradual fine-tuning, the routine ("felbo" or "howard")

4. The layers we want to fine-tune. 

For example

In [22]:
wide = Wide(wide_dim=np.unique(X_wide).shape[0], pred_dim=1)
deeptabular = TabResnet(
    blocks_dims=[128, 64, 32], 
    column_idx=tab_preprocessor.column_idx,
    embed_input=tab_preprocessor.embeddings_input,
    continuous_cols=continuous_cols)
model = WideDeep(wide=wide, deeptabular=deeptabular)

In [23]:
model

WideDeep(
  (wide): Wide(
    (wide_linear): Embedding(797, 1, padding_idx=0)
  )
  (deeptabular): Sequential(
    (0): TabResnet(
      (embed_layers): ModuleDict(
        (emb_layer_education): Embedding(17, 16, padding_idx=0)
        (emb_layer_native_country): Embedding(43, 16, padding_idx=0)
        (emb_layer_occupation): Embedding(16, 16, padding_idx=0)
        (emb_layer_relationship): Embedding(7, 8, padding_idx=0)
        (emb_layer_workclass): Embedding(10, 16, padding_idx=0)
      )
      (embedding_dropout): Dropout(p=0.0, inplace=False)
      (tab_resnet): DenseResnet(
        (dense_resnet): Sequential(
          (lin1): Linear(in_features=74, out_features=128, bias=True)
          (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (block_0): BasicBlock(
            (lin1): Linear(in_features=128, out_features=64, bias=True)
            (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

let's first train as usual

In [24]:
trainer4 = Trainer(model, objective="binary", metrics=[Accuracy])

In [25]:
trainer4.fit(X_wide=X_wide, X_tab=X_tab, target=target, val_split=0.1, n_epochs=2)

valid: 100%|██████████| 153/153 [00:01<00:00, 146.04it/s, loss=0.355, metrics={'acc': 0.8326}]


In [26]:
trainer4.save_model("models_dir/model2.t")

Now we are going to fine-tune the model components, and in the case of the `deeptabular` component, we will fine-tune the resnet-blocks and the linear layer but NOT the embeddings. 

For this, we need to access the model component's children: ``deeptabular`` $\rightarrow$ ``tab_resnet`` $\rightarrow$ ``dense_resnet`` $\rightarrow$ ``blocks``

In [27]:
model = trainer4.load_model("models_dir/model2.t")

In [28]:
tab_lin_layers = list(model.deeptabular.children())[1]

In [29]:
tab_deep_layers = list(
    list(list(list(model.deeptabular.children())[0].children())[2].children())[
        0
    ].children()
)[::-1][:2]

In [30]:
tab_layers = [tab_lin_layers] + tab_deep_layers

In [31]:
tab_layers

[Linear(in_features=32, out_features=1, bias=True),
 BasicBlock(
   (lin1): Linear(in_features=64, out_features=32, bias=True)
   (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (leaky_relu): LeakyReLU(negative_slope=0.01, inplace=True)
   (lin2): Linear(in_features=32, out_features=32, bias=True)
   (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (resize): Sequential(
     (0): Linear(in_features=64, out_features=32, bias=True)
     (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   )
 ),
 BasicBlock(
   (lin1): Linear(in_features=128, out_features=64, bias=True)
   (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (leaky_relu): LeakyReLU(negative_slope=0.01, inplace=True)
   (lin2): Linear(in_features=64, out_features=64, bias=True)
   (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (

And now simply

In [32]:
trainer5 = Trainer(model, objective="binary", metrics=[Accuracy])

In [33]:
trainer5.fit(
    X_wide=X_wide, 
    X_tab=X_tab, 
    target=target, 
    val_split=0.1, 
    finetune=True, 
    finetune_epochs=2, 
    finetune_deeptabular_gradual=True,
    finetune_deeptabular_layers = tab_layers,
    finetune_deeptabular_max_lr = 0.01,
    n_epochs=2)

  0%|          | 0/1374 [00:00<?, ?it/s]

Training wide for 2 epochs


  0%|          | 0/1374 [00:00<?, ?it/s]

Training deeptabular, layer 1 of 3


  0%|          | 0/1374 [00:00<?, ?it/s]

...

  0%|          | 0/1374 [00:00<?, ?it/s]

Training deeptabular, layer 3 of 3


  0%|          | 0/1374 [00:00<?, ?it/s]

Fine-tuning of individual components completed. Training the whole model for 2 epochs


valid: 100%|██████████| 153/153 [00:01<00:00, 144.21it/s, loss=0.343, metrics={'acc': 0.8415}]


Finally, there is one more use case I would like to consider. The case where we train only one component and we just want to fine-tune and stop the training afterwards, since there is no joined training. This is a simple as

In [43]:
deeptabular = TabMlp(mlp_hidden_dims=[200, 100], 
                   column_idx=tab_preprocessor.column_idx,
                   embed_input=tab_preprocessor.embeddings_input,
                   continuous_cols=continuous_cols)
model = WideDeep(deeptabular=deeptabular)

In [44]:
trainer6 = Trainer(model, objective="binary", metrics=[Accuracy])

In [45]:
trainer6.fit(X_wide=X_wide, X_tab=X_tab, target=target, val_split=0.1, n_epochs=1)

valid: 100%|██████████| 153/153 [00:00<00:00, 155.49it/s, loss=0.343, metrics={'acc': 0.8316}]


In [46]:
trainer6.save_model("models_dir/model3.t")

...times go by...

In [47]:
trainer7 = Trainer(model_path="models_dir/model3.t", objective="binary", metrics=[Accuracy])

In [49]:
trainer7.fit(
    X_wide=X_wide, 
    X_tab=X_tab, 
    target=target, 
    val_split=0.1, 
    finetune=True, 
    finetune_epochs=2,
    finetune_max_lr=0.01,
    stop_after_finetuning=True
    
)   

  0%|          | 0/1374 [00:00<?, ?it/s]

Training deeptabular for 2 epochs


epoch 2: 100%|██████████| 1374/1374 [00:11<00:00, 124.72it/s, loss=0.34, metrics={'acc': 0.8363}] 


Fine-tuning finished


In [50]:
import shutil

shutil.rmtree("models_dir/")